In [8]:
# %cd ../
%pwd
# should be /code

'/home/oli/Research/Joe/agent-goals/code'

In [6]:
from pgmpy.models import BayesianNetwork,  FactorGraph


'/home/oli/Research/Joe/agent-goals/code/scratch'

In [9]:
from pgmpy.readwrite import BIFReader
reader = BIFReader('pdg/data/alarm.bif')
reader.get_edges()

[['LVFAILURE', 'HISTORY'],
 ['LVEDVOLUME', 'CVP'],
 ['LVEDVOLUME', 'PCWP'],
 ['HYPOVOLEMIA', 'LVEDVOLUME'],
 ['LVFAILURE', 'LVEDVOLUME'],
 ['HYPOVOLEMIA', 'STROKEVOLUME'],
 ['LVFAILURE', 'STROKEVOLUME'],
 ['ERRLOWOUTPUT', 'HRBP'],
 ['HR', 'HRBP'],
 ['ERRCAUTER', 'HREKG'],
 ['HR', 'HREKG'],
 ['ERRCAUTER', 'HRSAT'],
 ['HR', 'HRSAT'],
 ['ANAPHYLAXIS', 'TPR'],
 ['ARTCO2', 'EXPCO2'],
 ['VENTLUNG', 'EXPCO2'],
 ['INTUBATION', 'MINVOL'],
 ['VENTLUNG', 'MINVOL'],
 ['FIO2', 'PVSAT'],
 ['VENTALV', 'PVSAT'],
 ['PVSAT', 'SAO2'],
 ['SHUNT', 'SAO2'],
 ['PULMEMBOLUS', 'PAP'],
 ['INTUBATION', 'SHUNT'],
 ['PULMEMBOLUS', 'SHUNT'],
 ['INTUBATION', 'PRESS'],
 ['KINKEDTUBE', 'PRESS'],
 ['VENTTUBE', 'PRESS'],
 ['MINVOLSET', 'VENTMACH'],
 ['DISCONNECT', 'VENTTUBE'],
 ['VENTMACH', 'VENTTUBE'],
 ['INTUBATION', 'VENTLUNG'],
 ['KINKEDTUBE', 'VENTLUNG'],
 ['VENTTUBE', 'VENTLUNG'],
 ['INTUBATION', 'VENTALV'],
 ['VENTLUNG', 'VENTALV'],
 ['VENTALV', 'ARTCO2'],
 ['ARTCO2', 'CATECHOL'],
 ['INSUFFANESTH', 'CATECHOL'],
 

In [11]:
M = reader.get_model()

In [12]:
M.adj

AdjacencyView({'HISTORY': {}, 'CVP': {}, 'PCWP': {}, 'HYPOVOLEMIA': {'LVEDVOLUME': {'weight': None}, 'STROKEVOLUME': {'weight': None}}, 'LVEDVOLUME': {'CVP': {'weight': None}, 'PCWP': {'weight': None}}, 'LVFAILURE': {'HISTORY': {'weight': None}, 'LVEDVOLUME': {'weight': None}, 'STROKEVOLUME': {'weight': None}}, 'STROKEVOLUME': {'CO': {'weight': None}}, 'ERRLOWOUTPUT': {'HRBP': {'weight': None}}, 'HRBP': {}, 'HREKG': {}, 'ERRCAUTER': {'HREKG': {'weight': None}, 'HRSAT': {'weight': None}}, 'HRSAT': {}, 'INSUFFANESTH': {'CATECHOL': {'weight': None}}, 'ANAPHYLAXIS': {'TPR': {'weight': None}}, 'TPR': {'CATECHOL': {'weight': None}, 'BP': {'weight': None}}, 'EXPCO2': {}, 'KINKEDTUBE': {'PRESS': {'weight': None}, 'VENTLUNG': {'weight': None}}, 'MINVOL': {}, 'FIO2': {'PVSAT': {'weight': None}}, 'PVSAT': {'SAO2': {'weight': None}}, 'SAO2': {'CATECHOL': {'weight': None}}, 'PAP': {}, 'PULMEMBOLUS': {'PAP': {'weight': None}, 'SHUNT': {'weight': None}}, 'SHUNT': {'SAO2': {'weight': None}}, 'INTUBATI

In [13]:
M.graph

{'name': 'Unknown'}

In [14]:
M.update

<bound method Graph.update of <pgmpy.models.BayesianNetwork.BayesianNetwork object at 0x7f73ebed9280>>

In [15]:
M.update()

NetworkXError: update needs nodes or edges input

In [17]:
M.nodes[0]

KeyError: 0

In [18]:
next(M.nodes)

TypeError: 'NodeView' object is not an iterator

In [19]:
next(iter(M.nodes))

'HISTORY'

In [20]:
from pgmpy.factors.discrete import TabularCPD

TabularCPD('grade', 3, )

In [28]:
from pdg import PDG
from pdg.rv import Variable as Var
from pdg.dist import CPT

M = PDG()
A = Var.alph("A",3);
B = Var.alph("B",2);
M += A, B

In [74]:
P = CPT.make_random(A, B); M += P

In [34]:
P.nfrom.name
hash("a")

-8014057836864215516

In [37]:
frozenset(P.nfrom)

frozenset({'a0', 'a1', 'a2'})

In [46]:
P

,b0,b1
a0,0.858183,0.141817
a1,0.919454,0.080546
a2,0.556172,0.443828


In [47]:
P.to_numpy().shape

(3, 2)

In [52]:
[a for a in P.nfrom.atoms]

[Var A {'a0', 'a1', 'a2'}]

In [63]:
tcpd = TabularCPD(P.nto.name, len(P.nto), P.to_numpy().T, [a.name for a in P.nfrom.atoms], [len(v) for v in P.nfrom.atoms])
tcpd.get_value(B=0, A=0)

0.8581831170498783

In [68]:
import numpy as np
V = {P.nfrom, P.nto, *P.nfrom.atoms}
V[0] == V[2]

TypeError: unhashable type: 'Variable'

In [69]:
%load_ext autoreload
%autoreload 2

In [77]:
print(P.as_(TabularCPD))

+-------+---------------------+--------------------+---------------------+
| A     | A(a0)               | A(a1)              | A(a2)               |
+-------+---------------------+--------------------+---------------------+
| B(b0) | 0.09701251401987165 | 0.5779129965171199 | 0.19557075362866636 |
+-------+---------------------+--------------------+---------------------+
| B(b1) | 0.9029874859801283  | 0.4220870034828801 | 0.8044292463713336  |
+-------+---------------------+--------------------+---------------------+


In [78]:
P

,b0,b1
a0,0.097013,0.902987
a1,0.577913,0.422087
a2,0.195571,0.804429


**Testing what happens if you add cycles / multiple cpds on same node**...

In [82]:
from pgmpy.models import BayesianNetwork

bn = BayesianNetwork([('A', 'B'), ('D','B')])
